In [49]:
from threading import Thread
import numpy as np
import pandas as pd
import pyshark
from tensorflow.keras.saving import load_model
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [50]:
df = pd.read_csv("test.csv")

In [51]:
df.head()

,Unnamed: 0,frame_info.encap_type,frame_info.time,frame_info.time_epoch,frame_info.number,frame_info.len,frame_info.cap_len,eth.type,ip.version,ip.hdr_len,...,tcp.flags.push,tcp.flags.ack,tcp.flags.urg,tcp.flags.cwr,tcp.window_size,tcp.checksum,tcp.urgent_pointer,tcp.options.mss_val,is_scan,length
0,0,1,"Jun 10, 2023 21:22:21.625151377 CEST",1.686425e+09,229,95,95,0x0800,4,20,...,1,1,0,0,501,0xbf61,0,0.0,0,95
1,0,1,"Jun 10, 2023 21:22:21.642060601 CEST",1.686425e+09,237,66,66,0x0800,4,20,...,0,1,0,0,683,0x8c05,0,0.0,0,66
2,0,1,"Jun 10, 2023 21:22:21.740360848 CEST",1.686425e+09,281,91,91,0x0800,4,20,...,1,1,0,0,683,0x645c,0,0.0,0,91
3,0,1,"Jun 10, 2023 21:22:21.740428100 CEST",1.686425e+09,282,66,66,0x0800,4,20,...,0,1,0,0,501,0x8bcd,0,0.0,0,66
4,0,1,"Jun 10, 2023 21:22:22.511846618 CEST",1.686425e+09,701,127,127,0x0800,4,20,...,1,1,0,0,501,0x9cf6,0,0.0,0,127


In [52]:
df.columns

Index(['Unnamed: 0', 'frame_info.encap_type', 'frame_info.time',
       'frame_info.time_epoch', 'frame_info.number', 'frame_info.len',
       'frame_info.cap_len', 'eth.type', 'ip.version', 'ip.hdr_len', 'ip.tos',
       'ip.id', 'ip.flags', 'ip.flags.rb', 'ip.flags.df', 'ip.flags.mf',
       'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.src',
       'ip.dst', 'ip.len', 'ip.dsfield', 'tcp.srcport', 'tcp.dstport',
       'tcp.seq', 'tcp.ack', 'tcp.len', 'tcp.hdr_len', 'tcp.flags',
       'tcp.flags.fin', 'tcp.flags.syn', 'tcp.flags.reset', 'tcp.flags.push',
       'tcp.flags.ack', 'tcp.flags.urg', 'tcp.flags.cwr', 'tcp.window_size',
       'tcp.checksum', 'tcp.urgent_pointer', 'tcp.options.mss_val', 'is_scan',
       'length'],
      dtype='object')

In [53]:
y_true = df['is_scan']
y_true

0      0
1      0
2      0
3      0
4      0
      ..
495    1
496    1
497    1
498    1
499    1
Name: is_scan, Length: 500, dtype: int64

In [54]:
df['Label'] = df['is_scan']

In [55]:
df

,Unnamed: 0,frame_info.encap_type,frame_info.time,frame_info.time_epoch,frame_info.number,frame_info.len,frame_info.cap_len,eth.type,ip.version,ip.hdr_len,...,tcp.flags.ack,tcp.flags.urg,tcp.flags.cwr,tcp.window_size,tcp.checksum,tcp.urgent_pointer,tcp.options.mss_val,is_scan,length,Label
0,0,1,"Jun 10, 2023 21:22:21.625151377 CEST",1.686425e+09,229,95,95,0x0800,4,20,...,1,0,0,501,0xbf61,0,0.0,0,95,0
1,0,1,"Jun 10, 2023 21:22:21.642060601 CEST",1.686425e+09,237,66,66,0x0800,4,20,...,1,0,0,683,0x8c05,0,0.0,0,66,0
2,0,1,"Jun 10, 2023 21:22:21.740360848 CEST",1.686425e+09,281,91,91,0x0800,4,20,...,1,0,0,683,0x645c,0,0.0,0,91,0
3,0,1,"Jun 10, 2023 21:22:21.740428100 CEST",1.686425e+09,282,66,66,0x0800,4,20,...,1,0,0,501,0x8bcd,0,0.0,0,66,0
4,0,1,"Jun 10, 2023 21:22:22.511846618 CEST",1.686425e+09,701,127,127,0x0800,4,20,...,1,0,0,501,0x9cf6,0,0.0,0,127,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,1,"Jun 10, 2023 21:22:23.010222290 CEST",1.686425e+09,1442,54,54,0x0800,4,20,...,1,0,0,0,0xc28c,0,0.0,1,54,1
496,0,1,"Jun 10, 2023 21:22:23.010222306 CEST",1.686425e+09,1443,54,54,0x0800,4,20,...,1,0,0,0,0x3844,0,0.0,1,54,1
497,0,1,"Jun 10, 2023 21:22:23.010222323 CEST",1.686425e+09,1444,54,54,0x0800,4,20,...,1,0,0,0,0x13c0,0,0.0,1,54,1
498,0,1,"Jun 10, 2023 21:22:23.010222338 CEST",1.686425e+09,1445,54,54,0x0800,4,20,...,1,0,0,0,0x2daa,0,0.0,1,54,1


In [56]:
df['frame_info.cap_len']

0       95
1       66
2       91
3       66
4      127
      ... 
495     54
496     54
497     54
498     54
499     54
Name: frame_info.cap_len, Length: 500, dtype: int64

In [57]:
df['tcp.dstport']

0        443
1      45810
2      45810
3        443
4        443
       ...  
495    47200
496    58158
497    59864
498    45958
499    46602
Name: tcp.dstport, Length: 500, dtype: int64

In [58]:
counts_df = df.groupby(['ip.src','tcp.dstport','Label']).agg(
    SYN_Flags_Count = ('tcp.flags.syn','count'), 
    RST_Flags_Count = ('tcp.flags.reset','count'),
    PSH_Flags_Count = ('tcp.flags.push','count'),
    CWE_Flags_Count = ('tcp.flags.cwr','count'),
    FIN_Flags_Count = ('tcp.flags.fin','count'),
    ACK_Flags_Count = ('tcp.flags.ack','count'),
    URG_Flags_Count = ('tcp.flags.urg','count'),
    Min_Packet_Lenght = ('length','min'),
    Max_Packet_Lenght = ('length','max'),
    Packet_Length_Mean = ('length','mean'),
    Packet_Length_Variance = ('length','var'),
    Packet_Length_Std = ('length','std'),
    Total_Fwd_Packets = ('length','count')
)

counts_df[['Packet_Length_Variance', 'Packet_Length_Std']] =  counts_df[['Packet_Length_Variance', 'Packet_Length_Std']].fillna(0.0)

In [59]:
counts_df

SYN_Flags_Count  RST_Flags_Count  \
ip.src          tcp.dstport Label                                     
162.159.138.234 50106       0                    2                2   
192.168.0.234   7           0                    1                1   
                17          0                    1                1   
                19          0                    1                1   
                20          0                    1                1   
...                                            ...              ...   
78.11.239.159   60326       1                    1                1   
                60352       1                    1                1   
                60694       1                    1                1   
                60968       1                    1                1   
                60984       1                    1                1   

                                   PSH_Flags_Count  CWE_Flags_Count  \
ip.src          tcp.dstport Label                                     
162.159.138.234 50106       0                    2                2   
192.168.0.234   7           0                    1                1   
                17          0                    1                1   
                19          0                    1                1   
                20          0                    1                1   
...                                            ...              ...   
78.11.239.159   60326       1                    1                1   
                60352       1                    1                1   
                60694       1                    1                1   
                60968       1                    1                1   
                60984       1                    1                1   

                                   FIN_Flags_Count  ACK_Flags_Count  \
ip.src          tcp.dstport Label                                     
162.159.138.234 50106       0                    2                2   
192.168.0.234   7           0                    1                1   
                17          0                    1                1   
                19          0                    1                1   
                20          0                    1                1   
...                                            ...              ...   
78.11.239.159   60326       1                    1                1   
                60352       1                    1                1   
                60694       1                    1                1   
                60968       1                    1                1   
                60984       1                    1                1   

                                   URG_Flags_Count  Min_Packet_Lenght  \
ip.src          tcp.dstport Label                                       
162.159.138.234 50106       0                    2                 66   
192.168.0.234   7           0                    1                 74   
                17          0                    1                 74   
                19          0                    1                 74   
                20          0                    1                 74   
...                                            ...                ...   
78.11.239.159   60326       1                    1                 54   
                60352       1                    1                 54   
                60694       1                    1                 54   
                60968       1                    1                 54   
                60984       1                    1                 54   

                                   Max_Packet_Lenght  Packet_Length_Mean  \
ip.src          tcp.dstport Label                                          
162.159.138.234 50106       0                    123                94.5   
192.168.0.234   7           0                     74                74.0   
         

In [60]:
counts_df = counts_df.reset_index()
counts_df = counts_df.drop(columns=['ip.src','tcp.dstport'])

In [61]:
df = counts_df.copy()

In [62]:
df.columns = df.columns.str.replace('_', ' ')

In [63]:
df

,Label,SYN Flags Count,RST Flags Count,PSH Flags Count,CWE Flags Count,FIN Flags Count,ACK Flags Count,URG Flags Count,Min Packet Lenght,Max Packet Lenght,Packet Length Mean,Packet Length Variance,Packet Length Std,Total Fwd Packets
0,0,2,2,2,2,2,2,2,66,123,94.5,1624.5,40.305087,2
1,0,1,1,1,1,1,1,1,74,74,74.0,0.0,0.000000,1
2,0,1,1,1,1,1,1,1,74,74,74.0,0.0,0.000000,1
3,0,1,1,1,1,1,1,1,74,74,74.0,0.0,0.000000,1
4,0,1,1,1,1,1,1,1,74,74,74.0,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,1,1,1,1,1,1,1,1,54,54,54.0,0.0,0.000000,1
481,1,1,1,1,1,1,1,1,54,54,54.0,0.0,0.000000,1
482,1,1,1,1,1,1,1,1,54,54,54.0,0.0,0.000000,1
483,1,1,1,1,1,1,1,1,54,54,54.0,0.0,0.000000,1


In [64]:
model = load_model('saved_model.h5')

2023-06-10 22:30:05.438446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 22:30:05.439806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 22:30:05.440282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [65]:
def preprocessing(df: pd.DataFrame):
    # Shuffle the dataset
    df = df.sample(frac=1)

    # Split features and labels
    x = df.iloc[:, df.columns != 'Label']
    y = df[['Label']].to_numpy()

    # Scale the features between 0 ~ 1
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)

    return x, y

In [76]:
def reshape_dataset_cnn(x: np.ndarray) -> np.ndarray:
    # Add padding columns
    result = np.zeros((x.shape[0], 81))
    #result[:, :-3] = x

    # Reshaping dataset
    result = np.reshape(result, (result.shape[0], 9, 9))
    result = result[..., tf.newaxis]
    return result

In [67]:
df.values.shape

(485, 14)

In [68]:
df.shape[0]

485

In [69]:
for i in range(81-13):
    df[f"{i}"] = np.zeros(df.shape[0])

In [70]:
df.head()

,Label,SYN Flags Count,RST Flags Count,PSH Flags Count,CWE Flags Count,FIN Flags Count,ACK Flags Count,URG Flags Count,Min Packet Lenght,Max Packet Lenght,...,58,59,60,61,62,63,64,65,66,67
0,0,2,2,2,2,2,2,2,66,123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,1,1,1,1,1,74,74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,1,1,1,1,74,74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,1,1,1,1,1,1,74,74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,1,1,1,1,1,1,74,74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
df.shape

(485, 82)

In [72]:
X, y = preprocessing(df)

In [73]:
X.shape

(485, 81)

In [77]:
X = reshape_dataset_cnn(X)

In [80]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

 1/16 [>.............................] - ETA: 0s

16/16 [==============================] - 0s 2ms/step


In [82]:

from sklearn.metrics import classification_report

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.58      1.00      0.73       281
           1       0.00      0.00      0.00       204

    accuracy                           0.58       485
   macro avg       0.29      0.50      0.37       485
weighted avg       0.34      0.58      0.43       485



/home/kbielecka/Documents/repos/Port-scan-detection-using-NN-SPZC/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kbielecka/Documents/repos/Port-scan-detection-using-NN-SPZC/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kbielecka/Documents/repos/Port-scan-detection-using-NN-SPZC/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [ ]:
# def evaluation(model: keras.Model, x_test: np.ndarray, y_test: np.ndarray):
#     score = model.evaluate(x_test, y_test, verbose=False)
#     logging.info('Evaluation:\nLoss: {}\nAccuracy : {}\n'.format(score[0], score[1]))

#     # F1 score
#     y_pred = model.predict(x_test, batch_size=1024, verbose=False)
#     y_pred = np.argmax(y_pred, axis=1)

#     logging.info("\n{}".format(classification_report(y_test, y_pred)))